In [1]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import imageio
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from IPython.display import display 
from PIL import Image
import tensorflow as tf
# import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import Dropout, Flatten,Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical


In [6]:
classes = os.listdir('flowers/flowers')[1:]

In [7]:
classes

['daisy', 'sunflower', 'rose', 'dandelion', 'tulip']

In [8]:
# create list for image arrays
X = []
# create list for y variable, class tracking 
y= []


for label in classes:
    direct = (f'flowers/flowers/{label}')
#     print(direct)
    for fl in os.listdir(f'{direct}'):
        if fl  != '.DS_Store':
            img_file = (f'{direct}/{fl}')
        # create image variable
        # resize image
            img = load_img(img_file,target_size=(224,224))
        # convert image to array
            img = img_to_array(img)

        # append image array to X list
            X.append(img)
        # append label to y list
            y.append(label)
        



In [9]:
# set X as numpy array
X=np.array(X)

In [10]:
# transoform y values to numbers
le = LabelEncoder()
le.fit(y)
y = le.transform(y)

In [27]:
# use train test split to obtain test set and use remainder for train validation split
X_rem, X_test, y_rem, y_test = train_test_split(X,y,test_size=.2)

In [28]:
# create train/validation split
X_train, X_val, y_train, y_val = train_test_split(X_rem,y_rem,test_size=.2)

In [29]:
# divide x variables by 255
X_train = X_train/255
X_val = X_val/255
X_test = X_test/255

In [30]:
# one hot encode y variables
y_train = to_categorical(y_train,num_classes=5)
y_val = to_categorical(y_val,num_classes=5)
y_test = to_categorical(y_test,num_classes=5)

In [12]:
# 
train_datagen = ImageDataGenerator(rescale=1./255,preprocessing_function=preprocess_input,validation_split=0.2)

In [16]:
# create train data generator
train_generator = train_datagen.flow(X_train,y_train, # path to the data folder
                                                 shuffle=True,
                                                 subset='training')

# create validation data generator
validation_generator = train_datagen.flow(X_train,y_train, # path to the data folder                   
                                                 batch_size=128,
                                                 shuffle=True,
                                                 subset='validation')



In [18]:
# get step size
step_size_train = train_generator.n//train_generator.batch_size

In [31]:
# Import the pretrained MobileNet model and do not include the final layer:
model = MobileNet(weights='imagenet', include_top=False)


# Assign the output of this base_model to a variable:
model = model.output
# add global pooling layer 
model = GlobalAveragePooling2D()(model)

# add 4 dense layers and a final layer of 5 for the 5 classes
model = Dense(243, activation='relu')(model) 
model = Dense(243, activation='relu')(model)
model = Dense(81, activation='relu')(model)
preds = Dense(5, activation='softmax')(model)

model = model(inputs = base_model.input, 
              outputs = preds)

In [33]:
# view summary of model
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128 

In [34]:
# set mobilenet model layers to be not trainable
for layer in model.layers[:87]:
    layer.trainable=False
    


In [35]:
# compile model using Adam optimizer
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [36]:
# fit model on train data and val data using 20 epochs
history = model.fit(X_train,y_train, 
                        epochs=20, batch_size=128,
                        validation_data=(X_val,y_val))

Train on 2766 samples, validate on 692 samples
Epoch 1/20
2766/2766 [==============================] - 3s 1ms/sample - loss: 0.9619 - acc: 0.6580 - val_loss: 0.9332 - val_acc: 0.6373
Epoch 2/20
2766/2766 [==============================] - 2s 670us/sample - loss: 0.3974 - acc: 0.8594 - val_loss: 0.6466 - val_acc: 0.7645
Epoch 3/20
2766/2766 [==============================] - 2s 672us/sample - loss: 0.2683 - acc: 0.9082 - val_loss: 0.6599 - val_acc: 0.7673
Epoch 4/20
2766/2766 [==============================] - 2s 679us/sample - loss: 0.2066 - acc: 0.9371 - val_loss: 0.8070 - val_acc: 0.7442
Epoch 5/20
2766/2766 [==============================] - 2s 676us/sample - loss: 0.1820 - acc: 0.9378 - val_loss: 0.6131 - val_acc: 0.7890
Epoch 6/20
2766/2766 [==============================] - 2s 672us/sample - loss: 0.1184 - acc: 0.9660 - val_loss: 0.7541 - val_acc: 0.7514
Epoch 7/20
2766/2766 [==============================] - 2s 673us/sample - loss: 0.0836 - acc: 0.9747 - val_loss: 0.7121 - val_a

In [38]:
# evaluate the model on test data
model.evaluate(X_test,y_test,batch_size=128)



865/865 [==============================] - 1s 749us/sample - loss: 0.7620 - acc: 0.8347


[0.7619696471732476, 0.8346821]